# Recap on previous hwk's functions defined:

In [19]:
import numpy as np
def Lj(r):
   
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

def LjTotal(positions):
    Energy=0
    N_atom = int(len(positions)/3)

    for i in range(N_atom - 1):
        for j in range(i + 1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1 - pos2)
            Energy = Energy + Lj(dist)
            
    return Energy
def init_pos(N, L=1):
    return L*np.random.random_sample((N*3,))

## Jacobian for the Leonard-Jones Potiential:
In the previous HWK, we allowed the minimize fucntion of SciPy to numerically calculate the gradient of our function based on its variables (the x,y,z positions of our atoms). This comes with a computational cost and should be utilized when the Jacobian (gradient at each point) cant be found analytically. Now we try to derive the analytical solution for the jacobian of the Leonard-Jones potential
The Jacobian can be defined as:

$$ \mathbf J = \left[ \begin{array} { c c c } { \frac { \partial f } { \partial x _ { 1 } } } & { \cdots } & { \frac { \partial f } { \partial x _ { n } } } \end{array} \right]$$

As we can see in our code above for the total Leonard-Jones potential, We add each two coupled paired of atom's Leonard-Jones Potentials for a total, This depends on the x,y,z of each atom leading our Energy Function to be:
$$ E(x_1, y_1, z_1, ..., x_n, y_n, z_n) = \sum_{i=1}^{N-1}\sum_{j=2}^{N}V(r_{ij})$$
where we defined $ r_{ij} $ as the distance between two atoms in space:
$$ r_{ij} = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2} $$

Now we calculate $ \mathbf J $ for a given as $x_{i}$ point from our coordinate set, (note this $x_{i}$ is different from our dummy counter 'i' variable in the sigma below):

$$ \mathbf J_i = \frac{\partial E}{\partial x_i} = \sum_{j=2}^{N}\frac{\partial V(r_{ij})}{\partial r_{ij}}\frac{\partial r_{ij}}{\partial x_i} $$



where $ \frac{\partial r_{ij}}{\partial x_i}$ is:
$$ \frac{\partial r_{ij}}{\partial x_i} = \frac{x_i- x_j}{r_{ij}}  $$
and $ \frac{\partial V(r_{ij})}{\partial r_{ij}}$ is:
$$ \frac{\partial V(r_{ij})}{\partial r_{ij}} = 24 \left( \frac{1}{r^7} - \frac{2}{r^{13}} \right)  $$


So now for every $x_{i}$ point in our coordiante set, we can define the analytical Jacobian as:
$$ \mathbf J_i = \frac{\partial E}{\partial x_i} = \sum_{i=1}^{N-1}\sum_{j=2}^{N} \left( \frac{24}{r_{ij}^7} - \frac{48}{r_{ij}^{13}} \right)\frac{x_i - x_j}{r_{ij}} $$
or to simplify for our code:
$$ \mathbf J_i = \frac{\partial E}{\partial x_i} = \sum_{i=1}^{N-1}\sum_{j=2}^{N} \left( \frac{24}{r_{ij}^8} - \frac{48}{r_{ij}^{14}} \right)\frac{x_i - x_j}{1} $$
### Note: Help and understanding for the jacobian code was provided by Dean and others in class  and with respect to the professors code

In [20]:
"We first define our first derivative with respect to r  "
def dLJ(r):
    
    r7 = r**7
    r13 = r7**5
    return 24*(1/r7 - 2/r13)


"Now we input our coordinate set, to find each repective Jacobian at that point"
def dLJdr(positions, dim = 3):
   
    N_atom = int(len(pos)/dim)
    
    F = np.zeros([N_atom, dim]) 
    
    for i in range(N_atom): # loop over each atom
        center_atom = positions[3*i:(3*i)+3]
        for j in range(N_atom): # loop over each neighboring atom
            neighbor_atom = positions[3*j:(3*j)+3] 
            if not any(center_atom == neighbor_atom): # ensures that center atom isn't its own neighbour (returns true if false)
                r = np.linalg.norm(center_atom-neighbor_atom) # find the radial distance between center atom and neighbor  
                tmp =  -48/r**14 + 24/r**8 
                Fxyz =  tmp * (center_atom-neighbor_atom) # calculate the force in x,y,z directions
                F[i] += Fxyz  # for all neighbors j, sum x,y,z forces
    """
    'tmp' is the dV/dr while the 'center_atom-neighbor_atom' calculates the rest of our jacobian at each
    point of (x1, y1, z1, ... , xn, yn, zn)
    """
    return F.flatten()

In [21]:
import time
from scipy.optimize import minimize

N = 4
f_values = []
pos_values =[]
N_runs = 100

t0 = time.perf_counter()
for i in range(N_runs):
    pos = init_pos(4)
    res = minimize(LjTotal, pos,jac=dLJdr, method='CG', tol=1e-4)
    #res = minimize(LjTotal, pos,jac=dLJdr, method='CG', tol=1e-4)
    f_values.append(res.fun)
    pos_values.append(res.x)
    print('\rRUN {:d} OUT OF {:d}: Total Energy = {:.4f}'.format(i, N_runs-1, res.fun), flush=True, end='')
t1 = time.perf_counter() # time end
time = t1 - t0
    
print('\nGround state energy:', min(f_values))
print("Total calculation time: ",time, "sec")

RUN 99 OUT OF 99: Total Energy = -6.0000
Ground state energy: -5.999999999999998
Total calculation time:  1.8443312090000745 sec


# Attempt at optimizing using the Evolution model :
Based on the documentation at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution

the differential_evolution function optimizes the LJTotal function below based on the cooridnate set of points as an input (position values for each atom ). The function requires a (min, max) bounds where each point is availiable to vary under. Based on our Init_pos function above, we defined out points (x,y,z) for each atom to be within a 1x1x1 box (L=1) above. So below, I defined the bounds for each bound to vary from (0,1) as a range where each coordinate can vary within. 

In [22]:
from scipy.optimize import differential_evolution
N=3
bounds=[]
for i in range(N):
    bounds.append((0,1))

def init_pos(N, L=1):
    return L*np.random.random_sample((N*3,))

pos = init_pos(N)
result = differential_evolution(LjTotal,bounds,args=pos)

result.fun

RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'